## Imports, Connect to Database

In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
#'/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

Connecting kui@ucsd-demo-db.datajoint.io:3306


./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD589'
yaml_file = 'shape_params-aws.yaml'
scripts_dir = os.environ['REPO_DIR']

## Create Process Table
--script  shapeolopy_code/scripts/Cell_generator.py

In [10]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    size_positive : int   #size of file
    size_negative : int   #size of file
    """
    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state
            s3_fp = (Structure & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+'CSHL_cells_dm/'+stack+'/'+ struc + '/'+stack+'_'+struc+'_'+state+'.pkl'
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [13]:
Process.drop()

`kui_diffusionmap`.`__process` (10 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [13]:
diffusion = Process()

In [17]:
diffusion.populate()#reserve_jobs=True

populating for  Sp5C
2019-05-31 06:43:27,344	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-31 06:43:27,345	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-31_06-43-27_344998_13875/logs.
2019-05-31 06:43:27,452	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:33284 to respond...
2019-05-31 06:43:27,561	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:22816 to respond...
2019-05-31 06:43:27,562	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-05-31 06:43:27,574	INFO node.py:511 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-31_06-43-27_344998_13875/logs.
2019-05-31 06:43:27,574	WARNING services.py:1293 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_m

/data/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))
/data/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_dm/MD589/
(pid=14334) VCA_positive 1340 0 / 66
(pid=14334) VCA_positive 11883 10 / 66
(pid=14334) VCA_positive 22470 20 / 66
(pid=14334) VCA_positive 26207 30 / 66
(pid=14334) VCA_positive 33562 40 / 66
(pid=14334) VCA_positive 49194 50 / 66
(pid=14334) VCA_positive 62190 60 / 66
(pid=14334) VCA_positive 64628
(pid=14334) VCA_positive finished in  70.6 seconds
upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_dm/MD589/VCA/MD589_VCA_positive.pkl to s3://mousebrainatlas-data/CSHL_cells_dm/MD589/VCA/MD589_VCA_positive.pkl
/data/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_dm/MD589/VCA/MD589_VCA_positive.pkl s3://mousebrainatlas-data/CSHL_cells_dm/MD589/VCA/MD589_VCA_positive.pkl


In [18]:
diffusion.fetch(as_dict=True)

[OrderedDict([('id', 0), ('size_positive', 1746), ('size_negative', 3095)]),
 OrderedDict([('id', 1), ('size_positive', 14), ('size_negative', 126)]),
 OrderedDict([('id', 2), ('size_positive', 3339), ('size_negative', 4785)]),
 OrderedDict([('id', 3), ('size_positive', 216), ('size_negative', 4851)]),
 OrderedDict([('id', 4), ('size_positive', 149), ('size_negative', 678)]),
 OrderedDict([('id', 5), ('size_positive', 975), ('size_negative', 2055)]),
 OrderedDict([('id', 6), ('size_positive', 2501), ('size_negative', 5549)]),
 OrderedDict([('id', 7), ('size_positive', 7877), ('size_negative', 5100)]),
 OrderedDict([('id', 8), ('size_positive', 988), ('size_negative', 2013)]),
 OrderedDict([('id', 9), ('size_positive', 2656), ('size_negative', 4221)]),
 OrderedDict([('id', 10), ('size_positive', 1227), ('size_negative', 2946)]),
 OrderedDict([('id', 11), ('size_positive', 637), ('size_negative', 2429)]),
 OrderedDict([('id', 12), ('size_positive', 7041), ('size_negative', 10830)]),
 Ord